In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

### Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

### Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [2]:
batch_size = 64

# Create Data Loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shpae of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shpae of y: torch.Size([64]) torch.int64


In [3]:
### Creating Models

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

#Define model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [4]:
### Optimizing the model Parameters

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


In [5]:
### single training loop

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        #compute loss
        pred = model(X)
        loss = loss_fn(pred, y)

        #backward
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1)* len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")  ##loss_width=7,float  5 string,d:10进制


In [10]:
### check the performance

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n ")
    

In [11]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-----------------------------
loss: 1.890875  [   64/60000]
loss: 1.849900  [ 6464/60000]
loss: 1.735275  [12864/60000]
loss: 1.794975  [19264/60000]
loss: 1.660897  [25664/60000]
loss: 1.623014  [32064/60000]
loss: 1.662954  [38464/60000]
loss: 1.560716  [44864/60000]
loss: 1.588566  [51264/60000]
loss: 1.490581  [57664/60000]
Test Error: 
 Accuracy: 62.4%, Avg loss: 1.503347
 
Epoch 2
-----------------------------
loss: 1.568778  [   64/60000]
loss: 1.524631  [ 6464/60000]
loss: 1.382412  [12864/60000]
loss: 1.463117  [19264/60000]
loss: 1.332255  [25664/60000]
loss: 1.337029  [32064/60000]
loss: 1.360004  [38464/60000]
loss: 1.287936  [44864/60000]
loss: 1.319512  [51264/60000]
loss: 1.221579  [57664/60000]
Test Error: 
 Accuracy: 63.7%, Avg loss: 1.245505
 
Epoch 3
-----------------------------
loss: 1.322134  [   64/60000]
loss: 1.295523  [ 6464/60000]
loss: 1.138849  [12864/60000]
loss: 1.245902  [19264/60000]
loss: 1.112734  [25664/60000]
loss: 1.144398  [32064/60000]
lo

In [12]:
### saving models

torch.save(model.state_dict(),"model.pth")
print("Save Pytorch Model State to model.pth")

Save Pytorch Model State to model.pth


In [13]:
### Loading models

model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [15]:
### use the model to make predictions

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}, Actual: "{actual}"') 

Predicted: "Ankle boot, Actual: "Ankle boot"
